In [1]:
# imports
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv('.env_aneslit')
from openai import OpenAI
import os 
from pathlib import Path
client = OpenAI()



See additional documentation here: https://drive.google.com/drive/folders/1Rk1kmDjIKS9mDUZFpx33K_4gTkVBNXCF

In [2]:
# provide general instructions to the assistant/thread as to it's role/personality
instructions = '''
You will be given a set of instructions for patients scheduled to undergo surgery in the San Francisco Bay Area. The average American reads at a 6th grade level. 
With this in mind, please modify the instructions you are given document to improve the readability of while maintaining accuracy and completeness.
'''

# this will be attached to the top of every prompt
header_instructions = '''
The average American reads at a 6th grade level. With this in mind, please modify the following document to improve the readability of while maintaining accuracy and completeness.
'''



In [3]:
# get AVSs from file system
def get_avs():
  unmodified_avs = []
  # Create a List of Documents from all of our files in the ./docs folder
  for file in os.listdir("avs_docs/unmodified/plain_text"):
    if file.endswith(".pdf"):
        doc_path = "avs_docs/unmodified/plain_text/" + file
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = "avs_docs/unmodified/plain_text/" + file
    elif file.endswith('.txt'):
        doc_path = "avs_docs/unmodified/plain_text/" + file
    elif file.endswith('.md'):
        doc_path = "avs_docs/unmodified/plain_text/" + file
    elif file.endswith('.html'):
        doc_path = "avs_docs/unmodified/plain_text/" + file
    avs_text = Path(doc_path).read_text()
    unmodified_avs = unmodified_avs + [avs_text]
  return unmodified_avs

unmodified_avs = get_avs()
print("extracted AVSs: " + str(len(unmodified_avs)))


extracted AVSs: 5


In [4]:
avs_dataframe = pd.DataFrame(columns=['avs_id', 'model', 'seed', 'timestamp', 'text'])


In [5]:
# unmodified rows 
df_unmod = pd.DataFrame(columns=['avs_id', 'model', 'text'])
df_unmod['seed'] = [0,0,0,0,0]
df_unmod['timestamp'] = ["Prior","Prior","Prior","Prior","Prior"]
df_unmod['avs_id'] = [0,1,2,3,4]
df_unmod['text'] = unmodified_avs
df_unmod['model'] = ['unmodified','unmodified','unmodified','unmodified','unmodified']

In [6]:
avs_dataframe = pd.concat([avs_dataframe,df_unmod])
avs_dataframe

,avs_id,model,seed,timestamp,text
0,0,unmodified,0,Prior,\nDepartment: ANESTHESIA PREOPERATIVE PROGRAM\...
1,1,unmodified,0,Prior,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...
2,2,unmodified,0,Prior,\nDepartment: ANESTHESIA PREOPERATIVE PROGRAM\...
3,3,unmodified,0,Prior,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...
4,4,unmodified,0,Prior,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...


In [8]:
# connect to database to store results
import duckdb
con = duckdb.connect('avs.db')
results = pd.DataFrame()
try:
    results = con.execute("SELECT * FROM avs_experiments_database").fetchdf()
except:
    pass
if results.empty:
    con.execute('CREATE TABLE avs_experiments_database AS SELECT * FROM avs_dataframe')
con.close()



In [12]:
# functions for conversion of raw AVS into more readable form 
import datetime

def get_timestamp():
    timestamp = '{:%Y-%m-%d-T-%H-%M-%S}'.format(datetime.datetime.now())
    return timestamp

def convert(avs_text, model, instructions, header_instructions,seed,temperature):
  full_prompt = header_instructions + "\n \n ---------------- \n \n " + avs_text
  completion = client.chat.completions.create(
    temperature=temperature,
    seed=seed,
    model=model,
    messages=[
      {"role": "system", "content": instructions },
      {"role": "user", "content": full_prompt}
    ]
  )
  return completion.choices[0].message

def convert_all_avs(unmodified_avs,model,instructions,header_instructions,temperature,num_replications):
  avs_df = pd.DataFrame(columns=['seed','text'])
  converted_avs = []
  seed = 0
  for replication_id in range(0,num_replications):
    avs_num = 0 
    for unmodified_avs_text in unmodified_avs:
      seed = replication_id * 100 + avs_num + 800
      print("---------------------------------------------------------")
      print(f"converting document {avs_num} with seed {seed}... \n")
      converted_avs_message = convert(unmodified_avs_text,model,instructions,header_instructions,seed,temperature)
      converted_avs_text = converted_avs_message.content
      print(f"converted text: ...{converted_avs_text[:50]}... \n")
      new_row = { 'seed': seed,
                  'text': converted_avs_text,
                  'timestamp': get_timestamp(),
                  'model': model,
                  'avs_id': avs_num }
     
      print("...saving to database")
      # save the results to a database
      con = duckdb.connect('avs.db')
      # add new row to the table
      keys_t = ", ".join(new_row.keys())
      vals_t = ", ".join(["?"]*len(new_row))
      query = f"INSERT INTO avs_experiments_database ({keys_t}) VALUES ({vals_t})"
      con.execute(query, list(new_row.values()))
      # close the connection
      con.close()
      
      # save to dataframe   
      avs_df = pd.concat([avs_df,pd.DataFrame([new_row])])
      avs_num = avs_num + 1
              
  return avs_df


In [10]:
# set temp (higher is more random)
temperature = 0.2
replications = 25

In [ ]:
avs35 = convert_all_avs(unmodified_avs,"gpt-3.5-turbo", instructions, header_instructions, temperature, replications)

In [ ]:
avs40 = convert_all_avs(unmodified_avs,"gpt-4-1106-preview",instructions, header_instructions, temperature, replications)

In [31]:
con = duckdb.connect('avs.db')
# add new row to the table
df = con.sql('SELECT * FROM avs_experiments_database').df() 
# close the connection
con.close()
print(df.query("model == 'unmodified'").avs_id.count())
print(df.query("model == 'gpt-3.5-turbo'").avs_id.count())
print(df.query("model == 'gpt-4-1106-preview'").avs_id.count())

5
125
125


In [32]:
# text analysis

# add new columns with correct data type
df['flesch_kincaid_grade'] = 99.9
df['flesch_reading_ease'] = 99.9
df['smog_index'] = 99.9
df['coleman_liau_index'] = 99.9
df['automated_readability_index'] = 99.9
df['dale_chall_readability_score'] = 99.9
df['difficult_words'] = 99.9
df['linsear_write_formula'] = 99.9
df['gunning_fog'] = 99.9
df['text_standard'] = "will be filled in"
df['fernandez_huerta'] = 99.9
df['szigriszt_pazos'] = 99.9
df['gutierrez_polini'] = 99.9
df['crawford'] = 99.9
df['gulpease_index'] = 99.9
df['osman'] = 99.9


df['diff_flesch_kincaid_grade'] = 99.9
df['diff_flesch_reading_ease'] = 99.9
df['diff_smog_index'] = 99.9
df['diff_coleman_liau_index'] = 99.9
df['diff_automated_readability_index'] = 99.9
df['diff_dale_chall_readability_score'] = 99.9
df['diff_difficult_words'] = 99.9
df['diff_linsear_write_formula'] = 99.9
df['diff_gunning_fog'] = 99.9
df['diff_text_standard'] = "cannot compute"
df['diff_fernandez_huerta'] = 99.9
df['diff_szigriszt_pazos'] = 99.9
df['diff_gutierrez_polini'] = 99.9
df['diff_crawford'] = 99.9
df['diff_gulpease_index'] = 99.9
df['diff_osman'] = 99.9


In [33]:
import textstat
for index, row in df.iterrows():
  


  df.loc[index, 'flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(row["text"])
  df.loc[index, 'flesch_reading_ease'] = textstat.flesch_reading_ease(row["text"])
  df.loc[index, 'smog_index'] = textstat.smog_index(row["text"])
  df.loc[index, 'coleman_liau_index'] = textstat.coleman_liau_index(row["text"])
  df.loc[index, 'automated_readability_index'] = textstat.automated_readability_index(row["text"])
  df.loc[index, 'dale_chall_readability_score'] = textstat.dale_chall_readability_score(row["text"])
  df.loc[index, 'difficult_words'] = textstat.difficult_words(row["text"])
  df.loc[index, 'linsear_write_formula'] = textstat.linsear_write_formula(row["text"])
  df.loc[index, 'gunning_fog'] = textstat.gunning_fog(row["text"])
  df.loc[index, 'text_standard'] = textstat.text_standard(row["text"])
  df.loc[index, 'fernandez_huerta'] = textstat.fernandez_huerta(row["text"])
  df.loc[index, 'szigriszt_pazos'] = textstat.szigriszt_pazos(row["text"])
  df.loc[index, 'gutierrez_polini'] = textstat.gutierrez_polini(row["text"])
  df.loc[index, 'crawford'] = textstat.crawford(row["text"])
  df.loc[index, 'gulpease_index'] = textstat.gulpease_index(row["text"])
  df.loc[index, 'osman'] = textstat.osman(row["text"])
  
  # get difference from basecase (unimproved)
  avs_id = row["avs_id"]
  basecase_row = df.query("model == 'unmodified' and avs_id == @avs_id")
  fkg = float(textstat.flesch_kincaid_grade(row["text"]))
  print(f"index {index} with FKG of {fkg}", end="...")

  
  df.loc[index, 'diff_flesch_kincaid_grade'] = float(textstat.flesch_kincaid_grade(row["text"])) - float(basecase_row['flesch_kincaid_grade'].iloc[0])
  df.loc[index, 'diff_flesch_reading_ease'] = float(textstat.flesch_reading_ease(row["text"])) - float(basecase_row['flesch_reading_ease'].iloc[0]) 
  df.loc[index, 'diff_smog_index'] = float(textstat.smog_index(row["text"])) - float(basecase_row['smog_index'].iloc[0]) 
  df.loc[index, 'diff_coleman_liau_index'] = float(textstat.coleman_liau_index(row["text"])) - float(basecase_row['coleman_liau_index'].iloc[0]) 
  df.loc[index, 'diff_automated_readability_index'] = float(textstat.automated_readability_index(row["text"])) - float(basecase_row['automated_readability_index'].iloc[0]) 
  df.loc[index, 'diff_dale_chall_readability_score'] = float(textstat.dale_chall_readability_score(row["text"])) - float(basecase_row['dale_chall_readability_score'].iloc[0]) 
  df.loc[index, 'diff_difficult_words'] = float(textstat.difficult_words(row["text"])) - float(basecase_row['difficult_words'].iloc[0]) 
  df.loc[index, 'diff_linsear_write_formula'] = float(textstat.linsear_write_formula(row["text"])) - float(basecase_row['linsear_write_formula'].iloc[0]) 
  df.loc[index, 'diff_gunning_fog'] = float(textstat.gunning_fog(row["text"])) - float(basecase_row['gunning_fog'].iloc[0]) 
  df.loc[index, 'diff_text_standard'] = "Cannot compute"
  df.loc[index, 'diff_fernandez_huerta'] = float(textstat.fernandez_huerta(row["text"])) - float(basecase_row['fernandez_huerta'].iloc[0]) 
  df.loc[index, 'diff_szigriszt_pazos'] = float(textstat.szigriszt_pazos(row["text"])) - float(basecase_row['szigriszt_pazos'].iloc[0]) 
  df.loc[index, 'diff_gutierrez_polini'] = float(textstat.gutierrez_polini(row["text"])) - float(basecase_row['gutierrez_polini'].iloc[0]) 
  df.loc[index, 'diff_crawford'] = float(textstat.crawford(row["text"])) - float(basecase_row['crawford'].iloc[0]) 
  df.loc[index, 'diff_gulpease_index'] = float(textstat.gulpease_index(row["text"])) - float(basecase_row['gulpease_index'].iloc[0]) 
  df.loc[index, 'diff_osman'] = float(textstat.osman(row["text"])) - float(basecase_row['osman'].iloc[0]) 
  
  



index 0 with FKG of 10.8...index 1 with FKG of 10.2...index 2 with FKG of 10.5...index 3 with FKG of 9.9...index 4 with FKG of 10.7...index 5 with FKG of 9.8...index 6 with FKG of 9.9...index 7 with FKG of 10.9...index 8 with FKG of 9.7...index 9 with FKG of 9.7...index 10 with FKG of 9.8...index 11 with FKG of 9.9...index 12 with FKG of 10.1...index 13 with FKG of 9.7...index 14 with FKG of 9.7...index 15 with FKG of 9.8...index 16 with FKG of 9.9...index 17 with FKG of 10.9...index 18 with FKG of 9.7...index 19 with FKG of 9.5...index 20 with FKG of 9.8...index 21 with FKG of 9.6...index 22 with FKG of 10.7...index 23 with FKG of 9.7...index 24 with FKG of 9.7...index 25 with FKG of 9.8...index 26 with FKG of 10.1...index 27 with FKG of 10.9...index 28 with FKG of 9.7...index 29 with FKG of 9.5...index 30 with FKG of 9.8...index 31 with FKG of 9.9...index 32 with FKG of 10.4...index 33 with FKG of 9.7...index 34 with FKG of 9.7...index 35 with FKG of 9.8...index 36 with FKG of 9.9...

In [34]:
df.to_excel("output/output.xlsx")